In [33]:
from sklearn.preprocessing import StandardScaler

In [34]:
class TrainDataset(Dataset):
    def __init__(self, df,CFG,train=True):
        self.df = df
        self.CFG = CFG

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        epoch = self.df["epoch"].values[idx]
        if epoch == 0:
            id_ = self.df["id"].values[idx]
            epoch = self.df["epoch"].values[idx]
            label = self.df["label"].values[idx]
            seq = np.load(f"data/array/{id_}_{epoch}.npy")#3000,7
            meta_data = self.df[['sleep_duration','night','sex', 'age','Resp oro-nasal','EMG submental','Temp rectal','Event marker']].values[idx]
            meta_data = torch.tensor(meta_data)
            seq = np.concatenate([seq[:,:7], seq[:,:7]], 0)
        else:
            try:
                id_ = self.df["id"].values[idx]
                epoch = self.df["epoch"].values[idx]
                label = self.df["label"].values[idx]
                seq = np.load(f"data/array/{id_}_{epoch}.npy")#3000,7
                pre_seq = np.load(f"data/array/{id_}_{epoch-1}.npy")#3000,7
                meta_data = self.df[['sleep_duration','night','sex', 'age','Resp oro-nasal','EMG submental','Temp rectal','Event marker']].values[idx]

                seq = np.concatenate([pre_seq[:,:7], seq[:,:7]], 0)
            except:
                id_ = self.df["id"].values[idx]
                epoch = self.df["epoch"].values[idx]
                label = self.df["label"].values[idx]
                seq = np.load(f"data/array/{id_}_{epoch}.npy")#3000,7
                meta_data = self.df[['sleep_duration','night','sex', 'age','Resp oro-nasal','EMG submental','Temp rectal','Event marker']].values[idx]
                meta_data = torch.tensor(meta_data)
                seq = np.concatenate([seq[:,:7], seq[:,:7]], 0)


        meta_data = torch.tensor(meta_data)
        if self.CFG.augmentation_crop_len>0:
            l = np.random.randint(self.CFG.augmentation_crop_len)
            r = np.random.randint(self.CFG.augmentation_crop_len)
            seq[:l,:] = 0
            seq[-r:,:] = 0

        seq = torch.from_numpy(seq).float()
        label = torch.tensor(label).long()

        return seq,label,meta_data

In [35]:

class TestDataset(Dataset):
    def __init__(self, df,CFG,train=True,transform1=None, transform2=None):
        self.df = df
        self.transform = transform1
        self.transform_ = transform2
        self.CFG = CFG

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        epoch = self.df["epoch"].values[idx]
        if epoch == 0:


            id_ = self.df["id"].values[idx]
            seq = np.load(f"data/array_test/{id_}_{epoch}.npy")#3000,7
            seq = np.concatenate([seq[:,:7], seq[:,:7]], 0)

        else:
            try:
                id_ = self.df["id"].values[idx]
                seq = np.load(f"data/array_test/{id_}_{epoch}.npy")#3000,7
                pre_seq = np.load(f"data/array_test/{id_}_{epoch-1}.npy")#3000,7
                seq = np.concatenate([pre_seq[:,:7], seq[:,:7]], 0)
            except:
                id_ = self.df["id"].values[idx]
                seq = np.load(f"data/array_test/{id_}_{epoch}.npy")#3000,7
                seq = np.concatenate([seq[:,:7], seq[:,:7]], 0)

        meta_data = self.df[['sleep_duration','night','sex', 'age','Resp oro-nasal','EMG submental','Temp rectal','Event marker']].values[idx]

        meta_data = torch.tensor(meta_data)
        seq = torch.from_numpy(seq).float()
        return seq, meta_data

In [36]:
#推論
def inference(model ,test_loader, device,CFG):
    model.to(device) 
    model.eval()
    probs = []
    scaler = torch.cuda.amp.GradScaler()
    softmax = nn.Softmax(dim = 1)

    for i, images in tqdm(enumerate(test_loader), total=len(test_loader)):
        images ,meta_data = images
        images = images.to(device)
        meta_data = meta_data.to(device)
        with torch.no_grad():
            y_preds = model(images.float(), meta_data.float())
            y_preds = softmax(y_preds)  
        probs.append(y_preds.to('cpu').numpy())

    probs = np.concatenate(probs)
    return probs

#dataAugmentation付きの推論で使うやつ
def cropaug(seq,crop_len):
    l = np.random.randint(crop_len)
    r = np.random.randint(crop_len)
    seq[:,:l,:] = 0
    seq[:,-r:,:] = 0
    return seq

#dataAugmentation付きの推論
def inference_tta(model, test_loader, device,CFG):
    

    model.to(device) 
    model.eval()
    probs = []

    for i, images in tqdm(enumerate(test_loader), total=len(test_loader)):
        images = images.to(device)
        with torch.no_grad():
            y_pred = []
            for i in range(2):
                if i==0:
                    augmented_image = images
                else:
                    augmented_image = cropaug(images,crop_len=CFG.augmentation.crop_len)

                y_preds,_ = model(augmented_image)
                softmax = nn.Softmax(dim = 1)
                y_preds = softmax(y_preds)
                y_pred.append(y_preds.to('cpu').numpy())
            y_pred = np.mean(np.stack(y_pred,axis=0), axis=0)
            
        probs.append(y_pred)

    probs = np.concatenate(probs)
    return probs

In [ ]:
cnn_features_size = {'efficientnet_b0':1280}

In [42]:
class CNN_add_metadata(nn.Module):
    def __init__(self, out_dim, CFG=CFG,n_meta_features=0, n_meta_dim=[512, 128], pretrained=False):
        super(CNN_add_metadata, self).__init__()
        self.n_meta_features = n_meta_features
        self.CFG = CFG
        self.efficientnet = timm.create_model(CFG.model_name, pretrained=True, num_classes=5)
        self.efficientnet.conv_stem = stft_conv(CFG)
        self.efficientnet.classifier=torch.nn.Identity()

        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])
        in_ch = cnn_features_size[CFG.model_name]
        if n_meta_features > 0:
            self.meta = nn.Sequential(
                nn.Linear(n_meta_features, n_meta_dim[0]),
                nn.BatchNorm1d(n_meta_dim[0]),
                nn.ReLU(),
                nn.Dropout(p=0.3),
                nn.Linear(n_meta_dim[0], n_meta_dim[1]),
                nn.BatchNorm1d(n_meta_dim[1]),
                nn.ReLU(),
            )
            in_ch += n_meta_dim[1]
        self.myfc = nn.Linear(in_ch, out_dim)#catした特徴量を入れる


    def forward(self, x, x_meta=None): 
        """  
        Parameters
        ----------
        x : cnn_features (batch×1280)
        x_meta : meata_features (batch×?)

        Returns
        ---------
        out: 
        """
        #print(x.shape)
        x = self.efficientnet(x)
        #print(x_meta.shape)
        if self.n_meta_features > 0:
            x_meta = self.meta(x_meta)
            x = torch.cat((x, x_meta), dim=1)
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                out = self.myfc(dropout(x))
            else:
                out += self.myfc(dropout(x))
        out /= len(self.dropouts)
        return out

In [46]:
def train_fn(CFG,fold,folds):
    torch.cuda.set_device(CFG.general_device)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"### fold: {fold} ###")
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index
    train_dataset = TrainDataset(folds.loc[trn_idx].reset_index(drop=True),train=True, 
                                 CFG=CFG)#
    val_folds = folds.loc[val_idx].reset_index(drop=True)
    valid_dataset = TrainDataset(folds.loc[val_idx].reset_index(drop=True),train=False,
                                CFG=CFG)#

    train_loader = DataLoader(train_dataset, batch_size=CFG.train_batch_size, shuffle=True, num_workers=8,drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.train_batch_size*2, shuffle=False, num_workers=8)

    ###  model select ============
    ## 系列モデル
    model = CNN_add_metadata(5,n_meta_features=8)
    model.to(device)

    ###  optim select ============
    if CFG.train_optim=="adam":
        optimizer = AdamW(model.parameters(), lr=CFG.train_lr, amsgrad=False)#CFG.train.lr
    elif CFG.train_optim=="sam":
        base_optimizer = torch.optim.SGD
        optimizer = SAM(model.parameters(),base_optimizer,lr=0.1, momentum=0.9)

    best_score = 0
    best_loss = np.inf
    best_acc=0
    best_preds = None

    softmax = nn.Softmax(dim = 1)
    scaler = torch.cuda.amp.GradScaler()
    step_train  = 0
    step_val = 0
    for epoch in range(CFG.train_epochs):
        start_time = time.time()
        model.train()
        avg_loss = 0.

        tk0 = tqdm(enumerate(train_loader), total=len(train_loader))

        for i, (images, labels, meta_data) in tk0:
            optimizer.zero_grad()

            images = images.to(device)
            labels = labels.to(device)
            meta_data = meta_data.to(device)
            rand = np.random.rand()
            if CFG.mixup>rand:
                images, y_a, y_b, lam = mixup_data(images, labels,alpha=2)
            with torch.cuda.amp.autocast():
                #cnn_features = model(images.float())
                y_preds = model(images.float(), meta_data.float())

                if CFG.mixup>rand:
                    train_loss = mixup_criterion(criterion, y_preds, y_a, y_b, lam)
                else:
                    train_loss = criterion(y_preds, labels)
            scaler.scale(train_loss).backward()
            scaler.step(optimizer)
            scaler.update()

        if CFG.train_scheduler_name!="none":
            scheduler.step()
            avg_loss += train_loss.item() / len(train_loader)

        model.eval()
        avg_val_loss = 0.
        preds = []
        valid_labels = []
        tk1 = tqdm(enumerate(valid_loader), total=len(valid_loader))

        for i, (images, labels, meta_data) in tk1:
            images = images.to(device)
            labels = labels.to(device)
            meta_data = meta_data.to(device)
            with torch.no_grad():

                y_preds = model(images.float(), meta_data.float())
                valid_loss = criterion(y_preds, labels)
            valid_labels.append(labels.to('cpu').numpy())
            y_preds = softmax(y_preds)
            preds.append(y_preds.to('cpu').detach().numpy())
            avg_val_loss += valid_loss.item() / len(valid_loader)

        preds = np.concatenate(preds)
        valid_labels = np.concatenate(valid_labels)

        #score = AUC(valid_labels,preds)
        score =0
        elapsed = time.time() - start_time
        log.info(f'  Epoch {epoch+1} - avg_train_loss: {avg_loss:.6f}  avg_val_loss: {avg_val_loss:.6f}  time: {elapsed:.0f}s')
        log.info(f'  Epoch {epoch+1} - AUC: {score:.6f}')
        th_preds = np.argmax(preds,axis=1)
        acc = accuracy_score(valid_labels,th_preds)
        log.info(f'  Epoch {epoch+1} - Acc: {acc:.4f}')
        if acc>best_acc:
            best_acc = acc
            best_preds = preds
            log.info(f'  Epoch {epoch+1} - Save Best Score: {avg_val_loss:.4f}-Accuracy:{acc:.4f}')
            torch.save(model.state_dict(), f'output/model_pth/fold{fold}_{CFG.general_exp_num}_model_baseline.pth')
        for i in range(5):
            col = f"pred_{i}"
            val_folds[col]=best_preds[:,i]
    return best_preds, valid_labels,val_folds


def submit(test,CFG,folds):
        print('run inference')
        torch.cuda.set_device(CFG.general_device)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        #test_dataset = TestDataset(test,CFG=CFG)
        #test_loader = DataLoader(test_dataset, batch_size=CFG.train_batch_size*2, shuffle=False)
        probs = []
        probs_rank = []
        for fold in range(5):
        # 標準化したいカラムを用意
            scaling_columns = ['sleep_duration',"night", "age", "sex","Resp oro-nasal","EMG submental","Temp rectal","Event marker"] 

            # trainと言うDataFrameにfit
            sc = StandardScaler().fit(folds[folds['fold']!=fold][scaling_columns])
            scaled_test = test.copy()
            # 標準化したカラムのみ元のDataFrameに戻す
            scaled_test[scaling_columns] = pd.DataFrame(sc.transform(scaled_test[scaling_columns]), columns=scaling_columns, index=scaled_test.index)
            #folds.update(scaled_test)
            test_dataset = TestDataset(scaled_test,CFG=CFG)
            test_loader = DataLoader(test_dataset, batch_size=CFG.train_batch_size*2, shuffle=False)
            weights_path = f'output/model_pth/fold{fold}_{CFG.general_exp_num}_model_baseline.pth'

            if CFG.model_name=="trans":
                model = TransformerModel(in_size=32,dim_feedforward=64,num_outputs=2,n_heads=1,n_encoders=2,pool ="avg",embed=True)
            elif CFG.model_name=="gru":
                model = gru_model(input_ch=7,pool = "avg")

            model = CNN_add_metadata(5,n_meta_features=8)
            state_dict = torch.load(weights_path,map_location=device)

            model.load_state_dict(state_dict)

            _probs = inference(model,test_loader, device,CFG)
            probs.append(_probs)
        probs = np.mean(probs, axis=0)
        if CFG.wandb:
            wandb.finish()
        return probs

In [48]:
def main(CFG) -> None:
    seed_torch(seed=42)

    folds = pd.read_csv("data/train_add_metadata_v4.csv")
    #train_record_df = pd.read_csv('sleep-stage-detection/train_records.csv')
    
    tmp = folds[folds["id"]=="5edb9d9"]
    tmp = tmp[tmp["epoch"]==1101]
    folds = folds.drop(index=tmp.index).reset_index(drop=True)
    
    test = pd.read_csv("data/test_add_metadata_v4.csv")
    test_record_df = pd.read_df = pd.read_csv('sleep-stage-detection/test_records.csv')

    preds = []
    valid_labels = []
    oof = pd.DataFrame()
    for fold in range(5):
        # ライブラリの読み込み
        from sklearn.preprocessing import StandardScaler

        # 標準化したいカラムを用意
        scaling_columns = ['sleep_duration',"night", "age", "sex","Resp oro-nasal","EMG submental","Temp rectal","Event marker"] 

        # trainと言うDataFrameにfit
        sc = StandardScaler().fit(folds[folds['fold']!=fold][scaling_columns])
        scaled_train = folds.copy()
        # 標準化したカラムのみ元のDataFrameに戻す
        scaled_train[scaling_columns] = pd.DataFrame(sc.transform(scaled_train[scaling_columns]), columns=scaling_columns, index=scaled_train.index)
        #folds.update(scaled_train)

        _preds, _valid_labels,_val_oof = train_fn(CFG,fold,scaled_train)
        preds.append(_preds)
        valid_labels.append(_valid_labels)
        oof = pd.concat([oof,_val_oof])

    pred = submit(test,CFG,folds)